<a href="https://colab.research.google.com/github/Vicente-Hernandez/simulador-evolucion-cartera/blob/main/simulador_evolucion_cartera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# =============================================================================
# PASO 1: INSTALACIÓN E IMPORTACIÓN DE LIBRERÍAS
# =============================================================================
!pip install pandas openpyxl numpy -q

import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')
np.random.seed(42)

# =============================================================================
# PASO 2: CLASE DE CONFIGURACIÓN PARA LA FASE 2
# =============================================================================
class ConfigFase2:
    """
    OBJETIVO: Centralizar los parámetros y reglas de la simulación de comportamiento.
    PAPEL: Define las probabilidades de los eventos que pueden ocurrir en la vida de un crédito.
    """
    # Probabilidad mensual de que cualquier cliente sufra un evento catastrófico.
    PROBABILIDAD_CISNE_NEGRO = 0.0005

    # Probabilidad de que un cliente se recupere, según su estado previo y arquetipo.
    PROBABILIDAD_RECUPERACION = {
        'Patrimonio Consolidado': {'de_atrasado': 0.90, 'de_default': 0.05},
        'Profesional Endeudado': {'de_atrasado': 0.60, 'de_default': 0.02},
        'Clase Media Estable': {'de_atrasado': 0.75, 'de_default': 0.03},
        'Clase Media Emergente': {'de_atrasado': 0.40, 'de_default': 0.01}
    }

    # Probabilidad mensual base de que un cliente tenga problemas (atraso), según su arquetipo.
    PROBABILIDAD_BASE_PROBLEMAS = {
        'Patrimonio Consolidado': 0.01,
        'Profesional Endeudado': 0.04,
        'Clase Media Estable': 0.02,
        'Clase Media Emergente': 0.05
    }

    # Parámetros para la lógica de tasación y renovación.
    DEPRECIACION_ANUAL_MIN = 0.08
    DEPRECIACION_ANUAL_MAX = 0.10
    AJUSTE_MANTENCION = 0.01 # Reducción de la depreciación por mantenciones.
    MARGEN_RETOMADOR_MIN = 0.05
    MARGEN_RETOMADOR_MAX = 0.10

# =============================================================================
# PASO 3: MOTOR DE SIMULACIÓN DE COMPORTAMIENTO Y RENOVACIÓN
# =============================================================================

def simular_pago_mensual(credito, estado_previo: str, contexto_macro: dict):
    """
    OBJETIVO: Simular el comportamiento de pago para un mes específico.
    PAPEL: Es el corazón probabilístico de la Fase 2, decidiendo si un cliente paga, se atrasa o se recupera.
    CÓMO: Ejecuta una secuencia de chequeos (cisne negro, recuperación, atraso) cuyas probabilidades
           son influenciadas por el perfil del cliente y el entorno macroeconómico.
    """
    arquetipo = credito['CLIENTE_ARQUETIPO']

    # 1. Shock Aleatorio ("Cisne Negro")
    if np.random.random() < ConfigFase2.PROBABILIDAD_CISNE_NEGRO:
        return 'En Default', np.random.randint(90, 120)

    # 2. Lógica de Recuperación
    if estado_previo == 'Atrasado':
        if np.random.random() < ConfigFase2.PROBABILIDAD_RECUPERACION[arquetipo]['de_atrasado']:
            return 'Al Día', 0
    if estado_previo == 'En Default':
        if np.random.random() < ConfigFase2.PROBABILIDAD_RECUPERACION[arquetipo]['de_default']:
            return 'Al Día', 0
        else:
            return 'En Default', 30

    # 3. Lógica de Atraso o Caída en Default
    prob_base = ConfigFase2.PROBABILIDAD_BASE_PROBLEMAS[arquetipo]
    multiplicador_riesgo = 2.5 if credito['FUTURO_DEFAULT'] else 1.0
    desempleo = contexto_macro.get('desempleo_tasa', 0.07)
    multiplicador_macro = 1 + (desempleo - 0.07) * 2
    prob_final_problemas = prob_base * multiplicador_riesgo * multiplicador_macro

    if np.random.random() < prob_final_problemas:
        if estado_previo == 'Atrasado':
            return 'En Default', 30
        else:
            return 'Atrasado', np.random.randint(5, 29)

    # 4. Si nada ocurre, paga a tiempo
    return 'Al Día', 0

def calcular_valor_retoma(credito_origen, mantenciones: bool):
    """
    OBJETIVO: Calcular el valor de tasación de un vehículo al final de su crédito.
    PAPEL: Determina el principal activo del cliente para una posible renovación.
    CÓMO: Aplica secuencialmente la deducción del IVA, la depreciación anual (ajustada por
           mantenciones) y el margen del revendedor.
    """
    precio_contado = credito_origen['PRECIO_FINAL']
    valor_sin_iva = precio_contado / 1.19
    años = credito_origen['PLAZO_MESES'] / 12

    tasa_depreciacion = np.random.uniform(ConfigFase2.DEPRECIACION_ANUAL_MIN, ConfigFase2.DEPRECIACION_ANUAL_MAX)
    if mantenciones:
        tasa_depreciacion -= ConfigFase2.AJUSTE_MANTENCION

    valor_depreciado = valor_sin_iva * ((1 - tasa_depreciacion) ** años)

    margen = precio_contado * np.random.uniform(ConfigFase2.MARGEN_RETOMADOR_MIN, ConfigFase2.MARGEN_RETOMADOR_MAX)
    valor_final_retoma = valor_depreciado - margen

    return int(max(0, valor_final_retoma))

def simular_decision_renovacion(credito, estado_final, pie_disponible):
    """
    OBJETIVO: Simular la decisión final del cliente (renovar o no).
    PAPEL: Aplica la lógica de negocio para determinar el resultado de los créditos con
           opción de renovación o los créditos tradicionales que terminan su ciclo.
    CÓMO: Compara el "pie disponible" (equity) del cliente con su pie original y evalúa
           su solvencia (arquetipo) para tomar una decisión probabilística.
    """
    if estado_final == 'En Default':
        return 'Inhabil_para_Renovar'

    pie_original = credito['PIE_MONTO']

    if credito['TIPO_FINANCIAMIENTO'] == 'Renovacion':
        if pie_disponible >= pie_original:
            return 'Renueva'
        else:
            pie_minimo_requerido = credito['PRECIO_FINAL'] * 0.20
            if pie_disponible < pie_minimo_requerido:
                es_solvente = credito['CLIENTE_ARQUETIPO'] in ['Patrimonio Consolidado', 'Profesional Endeudado']
                return 'Renueva' if es_solvente and np.random.random() < 0.65 else 'No Renueva'
            else:
                return 'Renueva'

    elif credito['TIPO_FINANCIAMIENTO'] == 'Tradicional':
        return 'Renueva' if pie_disponible >= (pie_original * 0.9) else 'No Renueva'

    return 'No Renueva'

def simular_evolucion_cartera(archivo_origen='dataset.xlsx'):
    """
    OBJETIVO: Orquestar la simulación completa de la Fase 2.
    PAPEL: Es la función principal que lee los datos, ejecuta la simulación mes a mes,
           gestiona el fin de ciclo de cada crédito y exporta los resultados finales.
    CÓMO: Itera sobre cada crédito aprobado de la Fase 1, simula su historial mensual y,
           al final del plazo, ejecuta la lógica de tasación y decisión de renovación.
    """
    print(f"Leyendo el archivo de origen '{archivo_origen}'...")
    try:
        df_origen = pd.read_excel(archivo_origen, sheet_name='Solo_Aprobados')
        df_financiado = df_origen[df_origen['TIPO_COMPRA'] == 'Financiado'].copy()
        df_financiado['ID_CREDITO'] = range(1, len(df_financiado) + 1)
        print(f"Se encontraron {len(df_financiado)} créditos financiados para simular.")
    except Exception as e:
        print(f"ERROR: No se pudo leer el archivo '{archivo_origen}'. Detalles: {e}")
        return

    transacciones_mensuales = []
    resultados_finales = []
    contexto_macro = {
        '2024-01':{'desempleo_tasa': 0.085}, '2024-02':{'desempleo_tasa': 0.088}, '2024-03':{'desempleo_tasa': 0.086},
        '2024-04':{'desempleo_tasa': 0.084}, '2024-05':{'desempleo_tasa': 0.082}, '2024-06':{'desempleo_tasa': 0.081},
        '2024-07':{'desempleo_tasa': 0.079}, '2024-08':{'desempleo_tasa': 0.077}, '2024-09':{'desempleo_tasa': 0.076},
        '2024-10':{'desempleo_tasa': 0.074}, '2024-11':{'desempleo_tasa': 0.072}, '2024-12':{'desempleo_tasa': 0.070},
        '2025-01':{'desempleo_tasa': 0.068}, '2025-02':{'desempleo_tasa': 0.066}, '2025-03':{'desempleo_tasa': 0.064},
        '2025-04':{'desempleo_tasa': 0.062}, '2025-05':{'desempleo_tasa': 0.060}, '2025-06':{'desempleo_tasa': 0.058},
        '2025-07':{'desempleo_tasa': 0.056}
    }

    for _, credito in tqdm(df_financiado.iterrows(), total=len(df_financiado), desc="Simulando Vida de Créditos"):
        saldo_pendiente = credito['MONTO_CAPITAL']
        estado_actual = 'Al Día'
        dias_atraso_acumulado = 0

        for i in range(1, int(credito['PLAZO_MESES']) + 1):
            if saldo_pendiente <= 0: break

            fecha_cuota = credito['FECHA_OPERACION'] + relativedelta(months=i)
            contexto_del_mes = contexto_macro.get(fecha_cuota.strftime('%Y-%m'), {'desempleo_tasa': 0.07})

            estado_pago, dias_atraso_mes = simular_pago_mensual(credito, estado_actual, contexto_del_mes)
            estado_actual = estado_pago

            if estado_pago in ['Atrasado', 'En Default']:
                dias_atraso_acumulado += dias_atraso_mes

            interes_mensual = saldo_pendiente * credito['TASA_MENSUAL']
            if estado_actual != 'En Default':
                capital_amortizado = credito['CUOTA_MENSUAL'] - interes_mensual
                saldo_pendiente -= capital_amortizado

            transacciones_mensuales.append({
                'ID_CREDITO': credito['ID_CREDITO'], 'NUMERO_CUOTA': i,
                'FECHA_CUOTA': fecha_cuota.strftime('%Y-%m-%d'), 'ESTADO_PAGO': estado_pago,
                'DIAS_ATRASO_MES': dias_atraso_mes, 'DIAS_ATRASO_ACUMULADO': dias_atraso_acumulado,
                'SALDO_PENDIENTE': int(max(0, saldo_pendiente))
            })

        # --- Lógica de Fin de Crédito y Renovación ---
        hizo_mantenciones = np.random.random() < {'Patrimonio Consolidado': 0.9, 'Profesional Endeudado': 0.7, 'Clase Media Estable': 0.8, 'Clase Media Emergente': 0.5}[credito['CLIENTE_ARQUETIPO']]
        valor_retoma = calcular_valor_retoma(credito, hizo_mantenciones)
        cuoton = credito['CUOTON_PORCENTAJE'] / 100 * credito['MONTO_CAPITAL'] if credito['TIPO_FINANCIAMIENTO'] == 'Renovacion' else 0
        pie_disponible = valor_retoma - cuoton
        decision_final = simular_decision_renovacion(credito, estado_actual, pie_disponible)

        resultados_finales.append({
            'ID_CREDITO': credito['ID_CREDITO'], 'ESTADO_PAGO_FINAL': estado_actual,
            'DIAS_ATRASO_ACUMULADOS_TOTAL': dias_atraso_acumulado, 'SALDO_PENDIENTE_FINAL': int(max(0, saldo_pendiente)),
            'HIZO_MANTENCIONES': hizo_mantenciones, 'VALOR_RETOMA_VEHICULO': valor_retoma,
            'PIE_DISPONIBLE_RENOVACION': int(pie_disponible), 'DECISION_FINAL': decision_final
        })

    print("\nSimulación completada.")
    df_evolucion = pd.DataFrame(transacciones_mensuales)
    df_resumen_final = pd.DataFrame(resultados_finales)
    df_final_enriquecido = pd.merge(df_financiado, df_resumen_final, on='ID_CREDITO', how='left')

    nombre_archivo_salida = 'dataset_evolucion.xlsx'
    print(f"Guardando resultados en '{nombre_archivo_salida}'...")
    try:
        with pd.ExcelWriter(nombre_archivo_salida) as writer:
            df_evolucion.to_excel(writer, sheet_name='Evolucion_Mensual', index=False)
            df_final_enriquecido.to_excel(writer, sheet_name='Estado_Final_Cartera', index=False)
        print(f"Éxito. El archivo ha sido guardado con 2 pestañas.")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

# =============================================================================
# PASO 4: ¡EJECUTAR LA SIMULACIÓN DE FASE 2!
# =============================================================================
if __name__ == '__main__':
    try:
        simular_evolucion_cartera()
    except Exception as e:
        print(f"ERROR FATAL: El proceso de simulación de Fase 2 no pudo completarse.")
        print(f"   Detalle: {e}")

Leyendo el archivo de origen 'dataset.xlsx'...
Se encontraron 3519 créditos financiados para simular.


Simulando Vida de Créditos: 100%|██████████| 3519/3519 [00:09<00:00, 378.26it/s]



Simulación completada.
Guardando resultados en 'dataset_evolucion.xlsx'...
Éxito. El archivo ha sido guardado con 2 pestañas.
